In [1]:
import sys

In [2]:
class Pizza:
    """
    A pizza of the input (.in) files.
    The id is the position in the file and the slices is the value on the file
    """

    def __init__(self, id: int, slices: int):
        self.id = id
        self.slices = slices

    def __str__(self):
        return f"({self.id}, {self.slices})"

In [3]:
class PizzaSet:
    """
    Data found on a (.in) file: the file, number of pizzas, maximum number of slices and the pizzas
    """

    def __init__(self, file:str):
        """
        Create and loads a file's information into a PizzaSet
        """
        self.file = file
        self.pizzas = None
        self.num_pizzas = 0
        self.max_slices = 0
        self._load()

    def _load(self):
        """
        Loads the file's information into a PizzaSet
        """
        with open(self.file) as f:
            for i, line in enumerate(f):
                spitted = line.split()
                if i == 0:
                    self.max_slices = int(spitted[0])
                    self.num_pizzas = int(spitted[1])
                elif i == 1:
                    self.pizzas = [Pizza(i, int(p)) for i, p in enumerate(spitted)]
                elif i > 1:
                    break
        self.pizzas.sort(key=lambda p: p.id)

    def write(self, order):
        """
        Writes a result into a file (.out)
        """
        file = self.file.replace("tests", "results")
        with open(file.replace("in","out"), 'w+') as f:        
            order_str = ""
            for pizza in order:
                order_str = order_str + f"{pizza.id} "   
            f.write(f"{len(order)}\n")
            f.write(f"{order_str}")

In [4]:
class PizzaSelector:
    """
    The implementation of the pizza selection algorithm
    """

    def __init__(self):
        self.order = None
        self.file = None
        self.pizza_set = None
        self.score = 0

    def _clear(self):
        """
        Clear selector's data of other executions
        """
        self.order = []
        self.score = 0

    def run(self, file: str):
        """
        1. calculate the number of slices left
        2. select the pizzas that will not be taken
            2.1. while there are slices to be removed and it has not been iterated n times
                 NOTE: n is the number of pizzas in the file -> with this number every pizza has the opportunity 
                    to be selected.
            2.2. select pizza with minimum error between pizza's slices and the current left slices
            2.3. update the slices_to_delete (current left slices) with the selected pizza
            2.4. add the pizza to pizzas_to_delete list and delete pizza from the PizzaSet (to not be choosen again)
        3. create the pizza order with all pizzas except the pizzas in the pizzas_to_delete list
        4. calcualte score with the pizza order list
        """
        # initialize
        self._clear()
        self.file = file
        self.pizza_set = PizzaSet(file)
        # calculate slices to delete
        total_number_of_slices = sum(p.slices for p in self.pizza_set.pizzas)
        slices_to_delete = total_number_of_slices - self.pizza_set.max_slices
        # obtain the pizzas to delete
        i = 0
        pizzas_to_delete = []
        while (slices_to_delete > 0) and (i < self.pizza_set.num_pizzas):
            i += 1
            selected_pizza = self._get_pizza_with_minimum_error(slices_to_delete)
            slices_to_delete -= selected_pizza.slices
            pizzas_to_delete.append(selected_pizza)
            self.pizza_set.pizzas.remove(selected_pizza)

        # create order with all pizzas except the pizzas to delete
        self.order = [p for p in self.pizza_set.pizzas if p not in pizzas_to_delete]
        # calculate number of slices (score)
        self.score = sum(p.slices for p in self.order)

    def _get_pizza_with_minimum_error(self, slices_to_delete):
        """
        Take the pizza that has the minimun difference in slices with the slices_to_delete
        """
        # initialize the select pizza to none and the error to the maximum value
        selected_pizza = None
        current_error = sys.maxsize
        # select the pizza with minimun errror in slices with the slices to delete
        for pizza in self.pizza_set.pizzas:
            if abs(slices_to_delete - pizza.slices) < current_error:
                selected_pizza = pizza
                current_error = abs(slices_to_delete - selected_pizza.slices)
        return selected_pizza

    def write(self):
        """
        Write the result to file (.out)
        """
        self.pizza_set.write(self.order)

In [5]:
def test(file):
    pizza_selector = PizzaSelector()
    pizza_selector.run(file)
    pizza_selector.write()
    print(f"{file}")
    
    print("\t percentage: {0:.7f}%".format(pizza_selector.score * 100 / pizza_selector.pizza_set.max_slices))
    print(f"\t score: {pizza_selector.score}")
    print(f"\t max number of slices: {pizza_selector.pizza_set.max_slices}")
    print(f"\t number of pizzas: {pizza_selector.pizza_set.num_pizzas}")
    print(f"\t number of ordered pizzas: {len(pizza_selector.order)}")
    print("=====================================================")
    
    return pizza_selector.score


In [6]:
    score = 0
    score += test('a_example.in')
    score += test('b_small.in')
    score += test('c_medium.in')
    score += test('d_quite_big.in')
    score += test('e_also_big.in')

    print("TOTAL SCORE: ", score)
    print("Is a record (> 1505004318): ", score > 1505004318)


a_example.in
	 percentage: 94.1176471%
	 score: 16
	 max number of slices: 17
	 number of pizzas: 4
	 number of ordered pizzas: 3
b_small.in
	 percentage: 98.0000000%
	 score: 98
	 max number of slices: 100
	 number of pizzas: 10
	 number of ordered pizzas: 5
c_medium.in
	 percentage: 99.9111111%
	 score: 4496
	 max number of slices: 4500
	 number of pizzas: 50
	 number of ordered pizzas: 48
d_quite_big.in
	 percentage: 99.9999989%
	 score: 999999989
	 max number of slices: 1000000000
	 number of pizzas: 2000
	 number of ordered pizzas: 1990
e_also_big.in
	 percentage: 99.9999570%
	 score: 504999783
	 max number of slices: 505000000
	 number of pizzas: 10000
	 number of ordered pizzas: 8172
TOTAL SCORE:  1505004382
Is a record (> 1505004318):  True
